# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv and preview
weather_df = pd.read_csv('../WeatherPy/output_data/cities.csv')
weather_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Elbistan,38.2059,37.1983,36.32,90,73,1.05,TR,1612899031
1,1,Okha,53.5739,142.9478,0.91,93,52,11.36,RU,1612899032
2,2,Mahébourg,-20.4081,57.7000,78.80,78,20,4.61,MU,1612899034
3,3,Laas,46.6166,10.7002,42.80,65,75,4.61,IT,1612899036
4,4,Ilulissat,69.2167,-51.1000,19.40,45,1,9.22,GL,1612899037
...,...,...,...,...,...,...,...,...,...,...
591,591,Tyul'gan,52.3405,56.1660,-20.18,88,28,2.06,RU,1612900274
592,592,Catuday,16.2923,119.8062,75.78,81,87,14.16,PH,1612900275
593,593,Tabas,33.5959,56.9244,54.45,41,0,1.50,IR,1612900115
594,594,Marawi,7.9986,124.2928,73.99,94,83,4.00,PH,1612900278


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps 
gmaps.configure(api_key=g_key)

In [35]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store humidity
humidity = weather_df["Humidity"]

# Set center and zoom
my_center = (locations.iloc[0]["Lat"], locations.iloc[0]["Lng"])
fig = gmaps.figure(zoom_level=2, center=my_center)


# Create and add layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

# Display
fig


Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [42]:
# Filter results by weather conditions
vacay_cities = weather_df[weather_df['Max Temp'] > 70]
vacay_cities = vacay_cities[vacay_cities['Max Temp'] < 80]
vacay_cities = vacay_cities[vacay_cities['Wind Speed'] < 10]
hotel_df = vacay_cities[vacay_cities['Cloudiness'] == 0]

hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,Salalah,17.0151,54.0924,77.00,53,0,2.30,OM,1612899043
70,70,Gouré,13.9835,10.2704,74.43,17,0,9.57,NE,1612899118
110,110,Pisco,-13.7000,-76.2167,77.00,69,0,6.91,PE,1612899166
156,156,East London,-33.0153,27.9116,72.86,74,0,4.43,ZA,1612899091
168,168,Bilma,18.6853,12.9164,71.19,18,0,8.25,NE,1612899240
230,230,Bara,10.3744,10.7288,78.44,16,0,9.31,NG,1612899322
432,432,Tessalit,20.1986,1.0114,70.92,17,0,9.86,ML,1612899574
550,550,Broken Hill,-31.9500,141.4333,73.40,27,0,6.91,AU,1612899901
557,557,Makkah al Mukarramah,21.4267,39.8261,75.22,50,0,6.02,SA,1612899910
567,567,Valle de Allende,26.9333,-105.4000,75.61,14,0,6.06,MX,1612899922


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:
# Copy df and add column
hotel_df = hotel_df.copy()
hotel_df["Hotel Name"] = ""

# Create params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "Lodging",
    "key": g_key
}

# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Lodging" and lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and print url to preview
    hotels = requests.get(base_url, params=params)
    #print(hotels.url)

    # Convert to json
    hotels = hotels.json()
    
   # Retreive hotel names
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
     
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")



In [44]:
# Drop NaNs and display 
hotel_df = hotel_df.dropna()
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,8,Salalah,17.0151,54.0924,77.00,53,0,2.30,OM,1612899043,Salalah
70,70,Gouré,13.9835,10.2704,74.43,17,0,9.57,NE,1612899118,Gouré
110,110,Pisco,-13.7000,-76.2167,77.00,69,0,6.91,PE,1612899166,Pisco
156,156,East London,-33.0153,27.9116,72.86,74,0,4.43,ZA,1612899091,East London
168,168,Bilma,18.6853,12.9164,71.19,18,0,8.25,NE,1612899240,Bilma
230,230,Bara,10.3744,10.7288,78.44,16,0,9.31,NG,1612899322,Bara
432,432,Tessalit,20.1986,1.0114,70.92,17,0,9.86,ML,1612899574,Achemelmel
550,550,Broken Hill,-31.9500,141.4333,73.40,27,0,6.91,AU,1612899901,Broken Hill
557,557,Makkah al Mukarramah,21.4267,39.8261,75.22,50,0,6.02,SA,1612899910,Mecca
567,567,Valle de Allende,26.9333,-105.4000,75.61,14,0,6.06,MX,1612899922,Valle de Allende


In [49]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]



In [51]:
# Add hotel locations

locations = hotel_df[["Lat", "Lng"]]

# Create, layer zoom and cetner 
my_center = (locations.iloc[8]["Lat"], locations.iloc[8]["Lng"])
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(zoom_level=2, center=my_center)

# Add layer to map
fig.add_layer(hotel_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [53]:
# Add marker layer ontop of heat map
my_center = (locations.iloc[8]["Lat"], locations.iloc[8]["Lng"])
fig = gmaps.figure(zoom_level=2, center=my_center)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)

# Add the layers to the map

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
fig.add_layer(markers)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))